In [1]:
import sqlite3 
import pandas as pd
#import pandas_profiling as pp
import numpy as np

import statsmodels.api as sm

import seaborn as sns

sns.set_style("whitegrid")

In [2]:
conn = sqlite3.connect('traffic.db')

In [3]:
sql_master = """SELECT * FROM sqlite_master"""
pd.read_sql_query(sql_master, conn)

,type,name,tbl_name,rootpage,sql
0,table,traffic,traffic,2,"CREATE TABLE traffic(\n ""SeqID"" TEXT,\n ""Dat..."
1,table,traffic_unique,traffic_unique,166571,"CREATE TABLE traffic_unique(\n SeqID TEXT,\n ..."


In [4]:
sql_all = "SELECT * FROM traffic"
df = pd.read_sql_query(sql_all, conn)
df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Driver City,Driver State,DL State,Arrest Type,Geolocation,Council Districts,Councils,Communities,Zip Codes,Municipalities
0,865defcf-8348-4e11-a5bf-996d8b3ead3c,06/03/2019,22:49:00,MCP,"2nd District, Bethesda",DRIVER CHANGING LANES WHEN UNSAFE,ROCKVILLE PIKE @ GROSVENOR LN,39.0231383333333,-77.102615,No,...,WASHINGTON,DC,DC,A - Marked Patrol,"(39.0231383333333, -77.102615)",1,1,12,111,1
1,20bff600-1843-46c4-9596-6b0670eb8d74,06/03/2019,22:48:00,MCP,"2nd District, Bethesda",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,RIVER/ROYAL DOMINION,38.99006,-77.15189,No,...,POTOMAC,MD,MD,A - Marked Patrol,"(38.99006, -77.15189)",1,1,12,103,1
2,e121eebe-daa1-4cca-afd4-8abb4b388ed6,06/03/2019,22:45:00,MCP,"4th District, Wheaton",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,GEORGIA @ EPPING,39.0662666666667,-77.0592166666667,No,...,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0662666666667, -77.0592166666667)",4,4,23,6,1
3,1e25ee9b-c019-41c2-9d74-41e05e6d1cee,06/03/2019,22:43:00,MCP,"4th District, Wheaton",STOP LIGHTS (*),GEORGIA AVE / ASPON HILL RD,39.0825783333333,-77.076885,No,...,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.0825783333333, -77.076885)",4,4,59,6,1
4,2c5730ec-d5e6-4657-a5a0-fcecca1ab140,06/03/2019,22:41:00,MCP,"2nd District, Bethesda",FAILURE TO DISPLAY & REFLECT RED COLOR LIGHT O...,ARLINGTON RD / BETHESDA AVE,38.97886,-77.0985183333333,No,...,BETHESDA,MD,MD,A - Marked Patrol,"(38.97886, -77.0985183333333)",1,1,12,111,1


In [5]:
pd.set_option("display.max_columns", 500)
df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,Fatal,Commercial License,HAZMAT,Commercial Vehicle,Alcohol,Work Zone,State,VehicleType,Year,Make,Model,Color,Violation Type,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation,Council Districts,Councils,Communities,Zip Codes,Municipalities
0,865defcf-8348-4e11-a5bf-996d8b3ead3c,06/03/2019,22:49:00,MCP,"2nd District, Bethesda",DRIVER CHANGING LANES WHEN UNSAFE,ROCKVILLE PIKE @ GROSVENOR LN,39.0231383333333,-77.102615,No,No,No,No,No,No,No,No,No,No,DC,02 - Automobile,2016,BMW,X3,SILVER,Warning,21-309(b),Transportation Article,false,BLACK,M,WASHINGTON,DC,DC,A - Marked Patrol,"(39.0231383333333, -77.102615)",1,1,12,111,1
1,20bff600-1843-46c4-9596-6b0670eb8d74,06/03/2019,22:48:00,MCP,"2nd District, Bethesda",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,RIVER/ROYAL DOMINION,38.99006,-77.15189,No,No,No,No,No,No,No,No,No,No,MD,02 - Automobile,2015,MERCEDEZBENZ,4S,BLACK,Warning,21-801.1,Transportation Article,false,WHITE,F,POTOMAC,MD,MD,A - Marked Patrol,"(38.99006, -77.15189)",1,1,12,103,1
2,e121eebe-daa1-4cca-afd4-8abb4b388ed6,06/03/2019,22:45:00,MCP,"4th District, Wheaton",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,GEORGIA @ EPPING,39.0662666666667,-77.0592166666667,No,No,No,No,No,No,No,No,No,No,MD,02 - Automobile,2015,HONDA,CIVIC,GRAY,Warning,21-801.1,Transportation Article,false,WHITE,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0662666666667, -77.0592166666667)",4,4,23,6,1
3,1e25ee9b-c019-41c2-9d74-41e05e6d1cee,06/03/2019,22:43:00,MCP,"4th District, Wheaton",STOP LIGHTS (*),GEORGIA AVE / ASPON HILL RD,39.0825783333333,-77.076885,No,No,No,No,No,No,No,No,No,No,MD,02 - Automobile,2014,KIA,4S,SILVER,ESERO,64*,,false,WHITE,M,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.0825783333333, -77.076885)",4,4,59,6,1
4,2c5730ec-d5e6-4657-a5a0-fcecca1ab140,06/03/2019,22:41:00,MCP,"2nd District, Bethesda",FAILURE TO DISPLAY & REFLECT RED COLOR LIGHT O...,ARLINGTON RD / BETHESDA AVE,38.97886,-77.0985183333333,No,No,No,No,No,No,No,No,No,No,MD,02 - Automobile,2008,ACUR,TL,GRAY,Warning,22-209(b),Transportation Article,false,WHITE,F,BETHESDA,MD,MD,A - Marked Patrol,"(38.97886, -77.0985183333333)",1,1,12,111,1


In [21]:
from scipy.stats import chisquare



In [22]:
df.Color.value_counts()

BLACK          311612
SILVER         279662
WHITE          237891
GRAY           172294
RED            119343
BLUE           111576
GREEN           54812
GOLD            46992
BLUE, DARK      32123
TAN             30408
MAROON          25856
BLUE, LIGHT     19699
BEIGE           17040
N/A             16362
GREEN, DK       15944
GREEN, LGT       8441
BROWN            6932
YELLOW           5684
ORANGE           5417
BRONZE           3477
PURPLE           2907
                 1746
MULTICOLOR       1157
CREAM             912
COPPER            469
PINK              211
CHROME             41
CAMOUFLAGE         28
Name: Color, dtype: int64

In [23]:
df.Accident.value_counts()

No     1482066
Yes      29172
2        11527
3         5255
4          676
5          340
Name: Accident, dtype: int64

In [24]:
contingency_table = pd.crosstab(
    df['Color'],
    df['Accident'],
    margins = True
)
contingency_table

Accident,2,3,4,5,No,Yes,All
Color,,,,,,,
,1,3,0,0,1738,4,1746
BEIGE,146,45,0,0,16469,380,17040
BLACK,2148,976,165,57,302626,5640,311612
BLUE,930,360,47,31,107938,2270,111576
"BLUE, DARK",240,176,11,12,30909,775,32123
"BLUE, LIGHT",213,47,4,5,19068,362,19699
BRONZE,21,4,0,0,3376,76,3477
BROWN,36,42,1,2,6730,121,6932
CAMOUFLAGE,0,0,0,0,28,0,28


In [10]:
f_obs = np.array([contingency_table['No'].values,
                  contingency_table['3'].values,
                  contingency_table['2'].values,
                  contingency_table['4'].values,
                  contingency_table['5'].values,
                  contingency_table['Yes'].values])
f_obs

array([[  1738,  16469, 302626, 107938,  30909,  19068,   3376,   6730,
            28,     41,    454,    889,  45472, 167286,  53114,  15397,
          8217,  24829,   1040,  16064,   5261,    211,   2721, 115165,
        271913,  29355, 230376,   5379],
       [     3,     45,    976,    360,    176,     47,      4,     42,
             0,      0,      0,      1,    145,    610,    169,     79,
            27,    160,     28,     39,      6,      0,      8,    434,
           937,    125,    796,     38],
       [     1,    146,   2148,    930,    240,    213,     21,     36,
             0,      0,      0,      7,    414,   1241,    337,    153,
            35,    253,     38,    150,     27,      0,     37,    994,
          1864,    271,   1899,     72],
       [     0,      0,    165,     47,     11,      4,      0,      1,
             0,      0,      0,      0,     14,     70,     18,      6,
             0,     12,      1,     16,      1,      0,     11,     75,
            7

In [25]:
from scipy import stats
stats.chi2_contingency(f_obs)[0:3]

(1584.9211972710643, 1.024847235881447e-245, 135)

The hypothesis: is accident type dependent on car color?  Our chi-square value was 3169, which stats that the two variables are independent with a significance level of 0.01 on 302 degrees of freedom.

In [26]:
contingency_table.iloc[:,4:6]

Accident,No,Yes
Color,,
,1738,4
BEIGE,16469,380
BLACK,302626,5640
BLUE,107938,2270
"BLUE, DARK",30909,775
"BLUE, LIGHT",19068,362
BRONZE,3376,76
BROWN,6730,121
CAMOUFLAGE,28,0


In [13]:
f_obs2 = np.array([contingency_table['No'].values,
                  contingency_table['Yes'].values])
stats.chi2_contingency(f_obs2)[0:3]

(669.4960836428701, 9.185234157078373e-124, 27)